In [11]:
import numpy as np
import scipy as spy
import matplotlib.pyplot as plt
import os
import librosa
import IPython.display as ipd

In [26]:
labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence", "unknown"]

In [27]:
print(labels)

['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'unknown']


In [28]:
dataset = np.load("yes.npy")
data = dataset[0]

First of all, we will load the training data into label.npy arrays and then use them. For this model we are using the mfcc array for training

In [29]:
x_train = []
y_train = []

#we will consider the mfcc foor x_train
k = []
for label in labels :
    data_label = np.load(label +".npy")
    print(data_label.shape)
    for arr in data_label :
#         print(arr.shape)
        mfcc_arr = librosa.feature.mfcc(arr)
        mfcc_arr = np.mean(mfcc_arr, axis=0)
#         print(mfcc_arr.shape)
        x_train.append(mfcc_arr)
        y_train.append(label)

(2377, 16000)
(2375, 16000)
(2375, 16000)
(2359, 16000)
(2353, 16000)
(2367, 16000)
(2367, 16000)
(2357, 16000)
(2380, 16000)
(2372, 16000)
(398, 16000)
(4096, 16000)


In [30]:
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)
np.save("x_train.npy", x_train)
np.save("y_train.npy", y_train)

(28176, 32)
(28176,)


In [31]:
from sklearn import model_selection
from sklearn import decomposition

we will try to apply PCA to extract the main features from our training data and then use them for training our model.

In [32]:
pca = decomposition.PCA(n_components = 12) 


In [33]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

In [34]:
x_train = pca.fit_transform(x_train)
print(x_train.shape)

(28176, 12)


In [35]:
from sklearn import preprocessing

Now we will perform label encoding on our given labels

In [36]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

In [37]:
print(y_train.shape)

(28176,)


We will now create a very simple neural network for our data consisting of 4 layers and train our model on it

In [38]:
from keras.callbacks import ModelCheckpoint 
from keras import layers
from keras.models import Sequential
from sklearn import model_selection
from datetime import datetime 

xtr, xte, ytr, yte = model_selection.train_test_split(x_train, y_train, test_size=0.2)
print(xtr.shape)
print(ytr.shape)
print(xte.shape)
print(yte.shape)
model = Sequential()
model.add(layers.Dense(256, activation='softmax', input_shape=(xtr.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(12, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


(22540, 12)
(22540,)
(5636, 12)
(5636,)


In [39]:
print(xtr.shape)
classifier = model.fit(xtr,ytr,epochs=100, batch_size=40)

(22540, 12)
Epoch 1/100
564/564 [==============================] - 1s 777us/step - loss: 2.4236 - accuracy: 0.1490
Epoch 2/100
564/564 [==============================] - 0s 760us/step - loss: 2.3149 - accuracy: 0.1755
Epoch 3/100
564/564 [==============================] - 0s 766us/step - loss: 2.2700 - accuracy: 0.1910
Epoch 4/100
564/564 [==============================] - 0s 744us/step - loss: 2.2413 - accuracy: 0.1979
Epoch 5/100
564/564 [==============================] - 0s 765us/step - loss: 2.2075 - accuracy: 0.2092
Epoch 6/100
564/564 [==============================] - 0s 795us/step - loss: 2.1916 - accuracy: 0.2175
Epoch 7/100
564/564 [==============================] - 0s 733us/step - loss: 2.1688 - accuracy: 0.2241
Epoch 8/100
564/564 [==============================] - 0s 779us/step - loss: 2.1535 - accuracy: 0.2353
Epoch 9/100
564/564 [==============================] - 0s 756us/step - loss: 2.1494 - accuracy: 0.2308
Epoch 10/100
564/564 [==============================] - 0s 75

In [40]:
# Evaluating the model on the training and testing set
score = model.evaluate(xtr, ytr, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(xte, yte, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 55.57%
Testing Accuracy: 17.16%


In [41]:
from sklearn.model_selection import StratifiedKFold

In [42]:
folds = StratifiedKFold(n_splits = 10, shuffle=True, random_state=17)

In [24]:
scores = []

for train, test in folds.split(x_train, y_train):
    model = Sequential()
    model.add(layers.Dense(256, activation='sigmoid', input_shape=(xtr.shape[1],)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    classifier = model.fit(xtr,ytr,epochs=100, batch_size=40)
    score = model.evaluate(xtr, ytr, verbose=0)
    scores.append(score[1]*100)
    
print(scores)

Epoch 1/100
564/564 [==============================] - 1s 846us/step - loss: 2.4280 - accuracy: 0.1410
Epoch 2/100
564/564 [==============================] - 0s 842us/step - loss: 2.3966 - accuracy: 0.1472
Epoch 3/100
564/564 [==============================] - 0s 874us/step - loss: 2.3917 - accuracy: 0.1525
Epoch 4/100
564/564 [==============================] - 1s 907us/step - loss: 2.3867 - accuracy: 0.1506
Epoch 5/100
564/564 [==============================] - 1s 923us/step - loss: 2.3794 - accuracy: 0.1576
Epoch 6/100
564/564 [==============================] - 1s 916us/step - loss: 2.3765 - accuracy: 0.1516
Epoch 7/100
564/564 [==============================] - 1s 955us/step - loss: 2.3708 - accuracy: 0.1553
Epoch 8/100
564/564 [==============================] - 1s 962us/step - loss: 2.3613 - accuracy: 0.1601
Epoch 9/100
564/564 [==============================] - 0s 798us/step - loss: 2.3575 - accuracy: 0.1630
Epoch 10/100
564/564 [==============================] - 0s 840us/step - l

KeyboardInterrupt: 

We can see that this model did not train very well and the highest accuracy achieved was 50% on the training set